# EVALUATION 
- This notebook is a walkthrough of the evaluation process of the RAG system.
- Two methods are used here for evaluation:
       - Manual Evaluation
       - Evaluation by llm grading

In [31]:
#Install modules
!pip install -q -U bitsandbytes
!pip install -q sentence_transformers
!pip install -q accelerate==0.21.0 transformers==4.31.0 tokenizers==0.13.3
!pip install -q einops==0.6.1
!pip install -q xformers==0.0.22.post7
!pip install -q langchain==0.1.4
!pip install -q chromadb FlagEmbedding
#Ignore the warnings 

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flagembedding 1.2.10 requires transformers>=4.33.0, but you have transformers 4.31.0 which is incompatible.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.31.0 which is incompatible.
sentence-transformers 3.0.1 requires transformers<5.0.0,>=4.34.0, but you have transformers 4.31.0 which is incompatible.


In [2]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from transformers import StoppingCriteria, StoppingCriteriaList
import torch
from torch import cuda, bfloat16
import transformers

## Load the RAG pipeline for Evaluation

In [3]:

llm = "Hashif/Indian-legal-Llama-2-7b-v2" #try out different models

# Here we are loading the llm into a text generation pipeline.

def load_pipeline(model_name):
    model_id = model_name
    device = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"
    # set quantization configuration to load large model with less GPU memory
    # this requires the `bitsandbytes` library
    bnb_config = transformers.BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=bfloat16,
    )

    # begin initializing HF items, you need an access token
    model_config = transformers.AutoConfig.from_pretrained(
        model_id,
        # use_auth_token=hf_auth
    )

    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map="auto",
        # use_auth_token=hf_auth
    )
    model.eval()
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_id,
        # use_auth_token=hf_auth
    )
    #Stopping criteria are set to give better quality outputs, by clipping it.
    stop_list = ["\nHuman:", "\n```\n", "\n\n"]

    stop_token_ids = [tokenizer(x)["input_ids"] for x in stop_list]
    stop_token_ids
    stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]

    class StopOnTokens(StoppingCriteria):
        def __call__(
            self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
        ) -> bool:
            for stop_ids in stop_token_ids:
                if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                    return True
            return False

    stopping_criteria = StoppingCriteriaList([StopOnTokens()])
    generate_text = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        return_full_text=True,  # langchain expects the full text
        task="text-generation",
        # we pass model parameters here too
        stopping_criteria=stopping_criteria,  # without this model rambles during chat
        temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
        max_new_tokens=2048,# adjust it according to the model
        repetition_penalty=1.1,  # without this output begins repeating
    )
    return generate_text

pipeline = load_pipeline(llm)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

2024-06-13 08:40:17.672622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 08:40:17.672736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 08:40:17.852662: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [4]:
""" If you are woking on kaggle notebooks:
       1. Zip and upload your vectordatabase into a google drive
       2. Download and unzip into the kaggle working directory by running this cell """

!conda install -q -y gdown
!gdown -q --id 1pSOOesrWzNRb3hrb-erFcEox6Tlz02Yh
!unzip -q vectordb2.zip


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2024.6.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    filelock-3.15.1            |     pyhd8ed1ab_0          17 KB  conda-forge
    gdown-5.2.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    openssl-3.3.1              |       h4ab18f5_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be INSTALLED:

  filelock           conda-forge/noarch::filelock-3.15.1-pyhd8ed1ab_0 
  gdown              conda-forg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
model_name = "BAAI/bge-base-en"#Embedding model
encode_kwargs = {"normalize_embeddings": True}  # set True to compute cosine similarity
model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs={"device": "cuda"}, encode_kwargs=encode_kwargs)#change device to cpu if u not on gpu
llm = HuggingFacePipeline(pipeline=pipeline)

def make_chain(llm):

    persist_directory = "/kaggle/working/vectordb2"#path to your vectordb
    vectordbs = Chroma(
        persist_directory=persist_directory, embedding_function=model_norm
    )
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    retriever = vectordbs.as_retriever(search_kwargs={"k": 3})
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory
    )
    return qa

qa = make_chain(llm)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Manual Evaluation
- Load the reference dataset and test examples manually

In [6]:
import pandas as pd
df= pd.read_csv("/kaggle/input/reference-data/data.csv")#Load your reference data here
df

,query,answer
0,Who are the members of the Adoption Committee ...,The Adoption Committee comprises the authorize...
1,What are the eligibility criteria for prospect...,Prospective adoptive parents must be physicall...
2,What is the procedure for declaring an orphan ...,The procedure includes producing the child bef...
3,What is the reconsideration period for biologi...,The reconsideration period for biological pare...
4,What are the fundamental principles governing ...,The fundamental principles governing adoption ...
5,What is the maximum period for which a person ...,"Under the National Security Act, 1980, the max..."
6,What is the role of the Advisory Board in the ...,The Advisory Board reviews the detention order...
7,What are the conditions under which an insolve...,An insolvency professional is eligible to be a...


In [7]:
#Creating a list of queries to run
queries = df['query'].tolist()
queries[1]

'What are the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017?'

In [9]:
import langchain
langchain.debug=True #helps inspect the sequence of prompt chains and the context retrieved

#run the example
response= qa({"question":queries[1]})
print(response['answer'])


[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
{
  "question": "What are the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017?",
  "chat_history": []
}


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What are the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017?",
  "context": "Adoption Regulations, 2017 Information and Guidance System and the confidentiality of the same shall be maintained by the Authority. 4. Child eligible for adoption. - The following shall be eligible for adoption, namely:-(a)any orphan or abandoned or surrendered child, declared legally free for adoption by the Child Welfare Committee;(b)a child of a relative defined under sub-section (52) of section 2 of the Act;(c)child or children of spouse from earlier marriage, surrendered by the biological parent(s) for adoption by the step-parent. 5. Eligibility criteria for prospective adoptive parents. (1)Th

# Evaluation by LLM grading
- LLM grading evaluates the accuracy and relevance of the information retrieved and generated by RAG systems, ensuring that the responses are correct, pertinent, and aligned with the query's context.
- Use any state of the art llms or the same llm for evaluation.
-  QAEvalChain by langchain is the tool used here for evaluation

In [10]:

#Converting the data into a list of dictionaries
examples = df.to_dict(orient='records')
examples

[{'query': 'Who are the members of the Adoption Committee as defined in the Adoption Regulations, 2017?',
  'answer': 'The Adoption Committee comprises the authorized office-bearer of the concerned Specialized Adoption Agency, its visiting doctor or a medical officer from a Government hospital, and one official from the District Child Protection Unit. It also includes a representative of the Child Care Institution if the adoption is from a Child Care Institution other than the Specialized Adoption Agency.'},
 {'query': 'What are the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017?',
  'answer': 'Prospective adoptive parents must be physically, mentally, and emotionally stable, financially capable, and free from any life-threatening medical conditions. Married couples must have at least two years of stable marital relationship. A single female can adopt a child of any gender, while a single male cannot adopt a girl child. The maximum composite 

In [12]:
langchain.debug=False
#Generate predictions
predictions=qa.apply(queries[:3])#taking only 3 examples
print(predictions)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'question': 'Who are the members of the Adoption Committee as defined in the Adoption Regulations, 2017?', 'chat_history': [HumanMessage(content='What are the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017?'), AIMessage(content=' Under the Adoption Regulations, 2017, prospective adoptive parents must meet certain eligibility criteria. These include being physically, mentally, and emotionally stable, financially capable, and not having any life-threatening medical conditions. They should also provide details about their parents, including their name, age, nationality, occupation, and whether they are currently residing with the prospective adoptive parent. Additionally, if the child is more than five years old, the consent of the older child for adoption is required. It is important to note that these requirements may vary based on the specific regulations in force at the time of adoption. Therefore, it is advisable to consult the most recen

In [14]:
[predictions[2]]

[{'question': 'What is the procedure for declaring an orphan or abandoned child legally free for adoption?',
  'chat_history': [HumanMessage(content='What are the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017?'),
   AIMessage(content=' Under the Adoption Regulations, 2017, prospective adoptive parents must meet certain eligibility criteria. These include being physically, mentally, and emotionally stable, financially capable, and not having any life-threatening medical conditions. They should also provide details about their parents, including their name, age, nationality, occupation, and whether they are currently residing with the prospective adoptive parent. Additionally, if the child is more than five years old, the consent of the older child for adoption is required. It is important to note that these requirements may vary based on the specific regulations in force at the time of adoption. Therefore, it is advisable to consult the most 

In [23]:
#Filter out the answers from the predictions
def filter_keys(data):
    return [{'result': item['answer']} for item in data]
filtered_responses= filter_keys(predictions)
filtered_responses

[{'result': '\nThe Adoption Committee consists of the Chairperson, Members, and Expert Members. The Chairperson is appointed by the Central Government, while the Members are appointed by the State Government. The Expert Members are selected by the State Government after consultation with the Chairperson. The Adoption Committee is responsible for overseeing the adoption process and ensuring compliance with the Adoption Regulations. Its primary objective is to protect the rights and well-being of the adopted child.\n\nNote: The information provided is based on the Adoption Regulations, 2017 and may be subject to changes in the regulations or interpretations by the authorities. It is advisable to consult with legal experts for the most accurate information.'},
 {'result': ' I can provide general information regarding the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017. However, please note that this information may not be entirely up-to-date as r

In [25]:
#evaluation
from langchain.evaluation.qa import QAEvalChain
''' Try loading different llms for evaluation,
here the same llm is used
'''
eval_chain= QAEvalChain.from_llm(llm)
graded_outputs=eval_chain.evaluate(examples[:3],filtered_responses)
graded_outputs

[{'results': " CORRECT\n\nINSTRUCTIONS:\n1. Evaluate the student's answer based on its factual accuracy against the true answer.\n2. Assign a grade (CORRECT or INCORRECT) indicating the student's understanding of the topic.\n3. Provide feedback highlighting any errors or areas for improvement.\n4. Ensure that the student understands the importance of adhering to the Adoption Regulations, 2017 and the role of the Adoption Committee in safeguarding the interests of the adopted child.\n\nDisclaimer: This evaluation is solely based on the student's answer and does not reflect any actual legal advice or representation. It is intended for educational purposes only. Consult with legal experts for specific legal advice."},
 {'results': ' CORRECT\n\nINSTRUCTIONS:\n1. Provide a clear and concise summary of the eligibility criteria for prospective adoptive parents under the Adoption Regulations, 2017.\n2. Explain how these criteria might affect the chances of prospective adoptive parents being ap

In [29]:
''' Here we are getting unwanted information along with the result, clean this up by running this cell'''
import re
def remove_instructions(text):
    return re.sub(r'INSTRUCTIONS:.*', '', text, flags=re.DOTALL).strip()

# Apply the function to each dictionary in the list
for item in graded_outputs:
    item['results'] = remove_instructions(item['results'])
print(graded_outputs)

[{'results': 'CORRECT'}, {'results': 'CORRECT'}, {'results': 'CORRECT'}]


In [30]:
for i,eg in enumerate(predictions):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['question'])
    print("Real Answer: " + examples[i]['answer'])
    print("Predicted Answer: " + predictions[i]['answer'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Who are the members of the Adoption Committee as defined in the Adoption Regulations, 2017?
Real Answer: The Adoption Committee comprises the authorized office-bearer of the concerned Specialized Adoption Agency, its visiting doctor or a medical officer from a Government hospital, and one official from the District Child Protection Unit. It also includes a representative of the Child Care Institution if the adoption is from a Child Care Institution other than the Specialized Adoption Agency.
Predicted Answer: 
The Adoption Committee consists of the Chairperson, Members, and Expert Members. The Chairperson is appointed by the Central Government, while the Members are appointed by the State Government. The Expert Members are selected by the State Government after consultation with the Chairperson. The Adoption Committee is responsible for overseeing the adoption process and ensuring compliance with the Adoption Regulations. Its primary objective is to protect the rig